## Building Data Frames

**Contents**
1. Cases
2. People
3. Acts/Bills
4. Speeches
5. Keywords? (maybe add to cases)

In [1]:
import pandas as pd
import regex as re
import numpy as np
import sqlite3

In [2]:
data = pd.read_csv(r"C:\Users\Avena Cheng\Desktop\Liberating Archives\data.csv")

In [3]:
data.head()

,Unnamed: 0,Title,Case No,Date,Year,Justices,Appearances,People
0,0,14 Penn Plaza LLC v. Pyett,No. 07-581,"December 1, 2008",2008,"['JUSTICE ALITO', 'JUSTICE BREYER', 'JUSTICE G...","PAUL SALVATORE, ESQ., New York, N.Y.; on behal...","PAUL SALVATORE, DAVID C. FREDERICK, CURTIS E. ..."
1,1,Abbott v. Perez,No. 17-586,"April 24, 2018",2018,"['JUSTICE ALITO', 'JUSTICE BREYER', 'JUSTICE G...","SCOTT A. KELLER, Solicitor General of Texas, ...","SCOTT A. KELLER, EDWIN S. KNEEDLER, MAX RENEA ..."
2,2,Abbott v. United States,No. 09-479,"October 4, 2010",2010,"['JUSTICE ALITO', 'JUSTICE BREYER', 'JUSTICE G...","DAVID L. HORAN, ESQ., Dallas, Texas; on behalf...","DAVID L. HORAN, JAMES E. RYAN"
3,3,Abdul-Kabir v. Quarterman,No. 05-11284,"January 17, 2006",2006,"['JUSTICE ALITO', 'JUSTICE BREYER', 'JUSTICE G...","ROBERT C. OWEN, ESQ., Austin, Tex.; on behalf...","ROBERT C. OWEN, EDWARD L. MARSHALL"
4,4,Abdur_Rahman v. Bell,No. 01-9094,"November 6, 2002",2002,['JUSTICE REHNQUIST'],"JAMES S. LIEBMAN, ESQ., New York, New York; on...","JAMES S. LIEBMAN, PAUL G. SUMMERS, PAUL J. ZID..."


## Cases DB

**Columns:**
* Case ID
* Title
* Case No ...notice there's some issues with this you should probably fix. 
* Date
* Year
* Month
* Day

In [4]:
data = data.drop('Unnamed: 0',axis=1)
data['Date']=pd.to_datetime(data[['Date']].stack()).unstack()

In [5]:
month = []
day = []
for i in range(len(data)):
    month += [data['Date'][i].month]
    day += [data['Date'][i].day]
data['Month'] = month
data['Day'] = day

In [6]:
case_df = data[['Title','Case No','Date','Year','Month','Day']]
new_cols = [i.lower() for i in case_df.columns]

In [7]:
new_cols[1] = "case_no"

In [8]:
case_df.columns = new_cols

In [17]:
import datetime as dt

In [25]:
case_df['date'] = case_df['date'].dt.normalize()

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [29]:
#case_df['date']

In [30]:
con = sqlite3.connect("database.db")

# Write the new DataFrame to a new SQLite table
case_df.to_sql("cases", con, if_exists="replace")

In [8]:
## not really useful
#roles = []
#for i in range(len(data)):
#    role = re.findall('[A-Z\s\.\,]+[A-Z][a-z]+\.',data[['Appearances']].iloc[i].values[0])
#    roles += role

In [9]:
len(set(data['Title']))

1302

## People DB

**Columns:**
* Person_ID
* Name
* Possibly role

In [10]:
data[['Appearances']].iloc[0].values[0]

'PAUL SALVATORE, ESQ., New York, N.Y.; on behalf of the Petitioners. DAVID C. FREDERICK, ESQ., Washington, D.C.; on behalf of the Respondents. CURTIS E. GANNON, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; on behalf of the United States, as amicus curiae, supporting the Respondents. '

In [11]:
data[data['Title'] == 'American Trucking Assns., Inc. v. Michigan Pub. Serv. Comm_n'][["Appearances"]].values

array([[' ROBERT DIGGES, JR., ESQ., Alexandria, Virginia; on behalf of the Petitioners . th Street, NW Suite ']],
      dtype=object)

In [12]:
# check for errors
for i in range(1302):
    lst = data['People'].iloc[i]
    if "P R O C E E D I N G S" in lst:
        print(data.iloc[i]['Title'])
    if "ARGUMENT" in lst:
        print(data.iloc[i]['Title'])
    if "JUSTCE" in lst:
        print(data.iloc[i]['Title'])
    if lst == "NW S":
        print(data.iloc[i]['Title'])

In [13]:
data['People'].iloc[0]

'PAUL SALVATORE, DAVID C. FREDERICK, CURTIS E. GANNON'

In [14]:
for i in range(1302):
    lst = data['People'].iloc[i]
    for j in lst:
        if j == "ANN O":
            print(lst)

In [15]:
everyone = []
for i in range(1302):
    lst = data['People'].iloc[i]
    lst = re.findall('[A-Z]+\s[A-Z\.\s]*[\'*A-Za-z\s*]+',data['People'].iloc[i])
    for i in lst:
        if "NW S" in i:
            continue
        if "JUSTICE" in i:
            continue
        everyone.append(i) 

In [16]:
data[['Title','People']].head()

,Title,People
0,14 Penn Plaza LLC v. Pyett,"PAUL SALVATORE, DAVID C. FREDERICK, CURTIS E. ..."
1,Abbott v. Perez,"SCOTT A. KELLER, EDWIN S. KNEEDLER, MAX RENEA ..."
2,Abbott v. United States,"DAVID L. HORAN, JAMES E. RYAN"
3,Abdul-Kabir v. Quarterman,"ROBERT C. OWEN, EDWARD L. MARSHALL"
4,Abdur_Rahman v. Bell,"JAMES S. LIEBMAN, PAUL G. SUMMERS, PAUL J. ZID..."


In [17]:
data['People'].iloc[0]

'PAUL SALVATORE, DAVID C. FREDERICK, CURTIS E. GANNON'

In [18]:
for i in range(1302):
    lst = data['People'].iloc[i]
    if "WORLDCOM N" in lst:
        print(data.iloc[i]['Title'])

Mathias v. Worldcom Technologies, Inc.


In [19]:
data[data['Title'] == "Mathias v. Worldcom Technologies, Inc." ]['Appearances'].values

array(['JOEL D. BERTOCCHI, ESQ., Solicitor General of Illinois, Chicago, Illinois; on behalf of the Petitioners. BARBARA B. McDOWELL, ESQ., Assistant to the Solicitor General, Department of Justice, Washington, D.C.; on behalf of the Respondent United States. PAUL M. SMITH, ESQ., Washington, D.C.; on behalf of the Respondents WorldCom Technologies, Inc., MCI WORLDCOM Network Services, Inc., MCImetro Access Services LLC, and Focal Communications Corp. '],
      dtype=object)

In [20]:
# double checking for consistency
for i in everyone:
    if "ANN O" in i:
        print(i)

ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL
ANN O'CONNELL


In [21]:
tgt = ", ".join(everyone)
print('The set being empty is a good thing. Means there are no weird names that have just a one letter last name')
set(re.findall("[A-Z]+\s[A-Z],",tgt))


The set being empty is a good thing. Means there are no weird names that have just a one letter last name


set()

In [22]:
all_justices =['GORSUCH','PHILLIPS','SCALIA','ROBERTS','KAVANAUGH','GINSBURG','BREYER','KENNEDY',
               'ALITO','STEVENS','KAGAN','REHNQUIST','THOMAS','SOTOMAYOR','SOUTER','BENNETT']
#roles = ['Justice','Justice','Justice','Justice','Justice','Justice','Justice','Justice','Justice','Justice','Justice','Justice','Justice','Justice']+roles
#len(roles)

In [23]:
re.findall('[A-Z]+\s[A-Z\.\s]*[A-Z]+',data[['People']].iloc[0][0])

['PAUL SALVATORE', 'DAVID C. FREDERICK', 'CURTIS E. GANNON']

In [24]:
people = all_justices+everyone
#people

### okay folks here's the cleaned people dataframe

In [25]:
people_df = pd.DataFrame({'Name':[],'Case':[]})
for i in range(1302):
    l = data['People'].iloc[i]
    title = data['Title'].iloc[i]
    l = re.findall('[A-Z]+\s[A-Z\.\s]*[\'*A-Za-z\s*]+',data['People'].iloc[i])
    for j in l:
        if "NW S" in j:
            continue
        if "JUSTICE" in j:
            continue
        people_df = people_df.append(pd.DataFrame({'Name':[j],'Case':[title]}))

In [26]:
people_df.reset_index().drop('index',axis=1).head(20)

,Name,Case
0,PAUL SALVATORE,14 Penn Plaza LLC v. Pyett
1,DAVID C. FREDERICK,14 Penn Plaza LLC v. Pyett
2,CURTIS E. GANNON,14 Penn Plaza LLC v. Pyett
3,SCOTT A. KELLER,Abbott v. Perez
4,EDWIN S. KNEEDLER,Abbott v. Perez
5,MAX RENEA HICKS,Abbott v. Perez
6,ALLISON J. RIGGS,Abbott v. Perez
7,DAVID L. HORAN,Abbott v. United States
8,JAMES E. RYAN,Abbott v. United States
9,ROBERT C. OWEN,Abdul-Kabir v. Quarterman


In [27]:
# Write the new DataFrame to a new SQLite table
people_df.to_sql("people", con, if_exists="replace")

